# Prompt Parallelization Workflow
In this notebook, we will build a content moderation system specifically designed for media and entertainment companies using the Prompt Parallelization pattern with CrewAI framework. This pattern allows us to process media content through multiple specialized LLM prompts in parallel and then aggregate the results.

At a high level, we'll be creating a media content moderation system that:

1. Takes content as input (scripts, user comments, social posts, etc.)
2. Analyzes it from multiple perspectives in parallel:
  - Explicit language detection and age-appropriate terminology
  - Inappropriate themes evaluation and cultural sensitivity
  - Misleading content identification and copyright concerns
  - Sentiment analysis and brand alignment assessment
3. Aggregates the results into a comprehensive moderation report tailored for media platforms


The following diagram depicts the workflow for a parallelization workflow implemented in Langgraph:

<img src="../../imgs/strands-parallel-llm-architecture.png" width="800"> 


## Import Required Libraries

Import the necessary CrewAI Flow components and Pydantic for data modeling. These libraries will enable us to create the parallel processing workflow and define structured data models for our content analysis agents.


In [ ]:
from crewai.flow.flow import Flow, listen, and_, start
from pydantic import BaseModel
from crewai import LLM

## Configure Logging

Set up logging configuration to track the execution of our content analysis workflow. This will help us monitor the process and debug any issues that might arise during the parallel analysis execution.


In [ ]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

## Define Agent State Model

Create a Pydantic data model to represent the state that will be shared across all analysis agents. This includes the input content and fields to store the results from each specialized analysis agent (explicit language, inappropriate themes, misleading content, and sentiment analysis).


In [ ]:
class AgentState(BaseModel):
    content: str = None
    explicitLanguageAnalysis: str = ""
    inappropriateThemeAnalysis: str = ""
    misleadingContentAnalysis: str= ""
    sentimentAnalysis: str = ""
    final_result: str = ""


## Content Analysis Parallel Flow Implementation

This is the core implementation of our parallel content analysis workflow using CrewAI Flow. The class defines four specialized analysis agents that run in parallel:

1. **Explicit Language Analysis Agent** - Analyzes content for profanity, age-appropriate language, and provides age rating recommendations
2. **Inappropriate Theme Analysis Agent** - Identifies problematic themes, cultural sensitivity issues, and regional content concerns  
3. **Misleading Content Analysis Agent** - Detects factual inaccuracies, potential copyright issues, and content requiring legal review
4. **Sentiment Analysis Agent** - Evaluates emotional tone, brand alignment, and potential audience impact

After all four agents complete their analysis, an **Aggregator** combines all results into a comprehensive moderation report with platform-specific recommendations for streaming services, broadcast TV, social media, and international distribution.


In [ ]:
class ContentAnalysisParallelFlow(Flow[AgentState]):

    @start()
    def start_method(self):
        pass

    @listen(start_method)
    def explicitLanguageAnalysisAgent(self):
        system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying explicit language and age-appropriate content.

# Analyze the following content for:

1. Explicit language and profanity
2. Age-appropriate terminology
3. Content suitable for different audience age ratings (G, PG, PG-13, R, etc.)
4. Dialogue or descriptions that may require parental guidance

## Provide a detailed report with:

1. Overall assessment (Family-Friendly/Teen-Appropriate/Mature/Adult-Only)
2. Specific instances of explicit language or age-inappropriate content found (if any)
3. Age rating recommendation (G, PG, PG-13, R, NC-17, etc.)
4. Recommendations for content revision to meet specific audience age targets

Format your response as JSON:

   {
      "assessment": "Family-Friendly/Teen-Appropriate/Mature/Adult-Only",
      "instances": ["example1", "example2"],
      "age_rating": "PG-13",
      "revision_recommendations": "Your recommendations here"
   }
"""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": self.state.content}
        ]
        llm = LLM(model="bedrock/us.amazon.nova-micro-v1:0")
        
        response = llm.call(messages=messages)
        self.state.explicitLanguageAnalysis = response
        
    @listen(start_method)
    def inappropriateThemeAnalysisAgent(self):
        system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying inappropriate themes and cultural sensitivity issues.

# Analyze the following content for:

1. Inappropriate themes such as violence, discrimination, or harmful ideologies
2. Cultural sensitivity concerns across global audiences
3. Potentially offensive stereotypes or representations
4. Content that may be inappropriate for certain regions or markets


## Provide a detailed report with:

1. Overall assessment (Universally Acceptable/Regionally Sensitive/Potentially Offensive)
2. Specific inappropriate themes or cultural sensitivity issues identified (if any)
3. Regions or demographics that may find the content problematic
4. Recommendations for content revision to improve cultural sensitivity

# Format your response as JSON:

   {
      "assessment": "Universally Acceptable/Regionally Sensitive/Potentially Offensive",
      "themes": ["theme1", "theme2"],
      "sensitive_regions": ["region1", "region2"],
      "revision_recommendations": "Your recommendations here"
   }
"""
        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": self.state.content}
            ]
        llm = LLM(model="bedrock/us.amazon.nova-micro-v1:0")
        response = llm.call(messages=messages)
        self.state.inappropriateThemeAnalysis = response

    @listen(start_method)
    def misleadingContentAnalysisAgent(self):
        system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying misleading content, factual inaccuracies, and potential copyright issues.

# Analyze the following content for:

1. Misleading claims or factual inaccuracies
2. Potential copyright or intellectual property concerns
3. Unauthorized references to protected brands, characters, or properties
4. Content that may require fact-checking or legal review

## Provide a detailed report with:

1. Overall assessment (Factually Sound/Needs Verification/Potentially Infringing)
2. Specific misleading claims or factual inaccuracies identified (if any)
3. Potential copyright or IP concerns
4. Recommendations for content revision to address factual or legal issues

Format your response as JSON:

   {
      "assessment": "Factually Sound/Needs Verification/Potentially Infringing",
      "inaccuracies": ["inaccuracy1", "inaccuracy2"],
      "copyright_concerns": ["concern1", "concern2"],
      "revision_recommendations": "Your recommendations here"
   }
""" 
        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": self.state.content}
            ]
        llm = LLM(model="bedrock/us.amazon.nova-lite-v1:0")
        response = llm.call(messages=messages)
        self.state.misleadingContentAnalysis = response

    @listen(start_method)
    def sentimentAnalysisAgent(self):
        system_prompt = """You are a content moderator for a major media and entertainment company specializing in sentiment analysis and brand alignment.

# Analyze the following content for:

1. Overall emotional tone and sentiment
2. Alignment with typical media brand values (positivity, inclusivity, creativity, etc.)
3. Potential impact on audience perception and brand reputation
4. Emotional resonance with target demographics

# Provide a detailed report with:
1. Overall sentiment (Positive/Neutral/Negative)
2. Brand alignment assessment (Aligned/Partially Aligned/Misaligned)
3. Potential emotional impact on viewers/readers
4. Recommendations for content revision to improve brand alignment

Format your response as JSON:

{
    "sentiment": "Positive/Neutral/Negative",
    "brand_alignment": "Aligned/Partially Aligned/Misaligned",
    "emotional_impact": "Your assessment here",
    "revision_recommendations": "Your recommendations here"
}
"""
        messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": self.state.content}
                ]
        llm = LLM(model="bedrock/us.amazon.nova-lite-v1:0")
        response = llm.call(messages=messages)
        self.state.sentimentAnalysis = response

    @listen(and_(explicitLanguageAnalysisAgent, inappropriateThemeAnalysisAgent, misleadingContentAnalysisAgent, sentimentAnalysisAgent))
    def aggregator(self):
        system_prompt = """You are a senior content moderator for a major media and entertainment company responsible for creating comprehensive moderation reports with platform-specific recommendations.

   You are given the moderation analysis reports as described below:
   
   - Explicit Language & Age-Appropriate Analysis
   - Inappropriate Themes & Cultural Sensitivity Analysis
   - Misleading Content & Copyright Analysis
   - Sentiment & Brand Alignment Analysis


   Based on these reports, provide:
   1. Overall moderation decision (Approve/Revise/Reject)
   2. Summary of key concerns from all analyses
   3. Platform-specific recommendations for:
      - Streaming platforms (Netflix, Disney+, Prime Video, etc.)
      - Broadcast television
      - Social media platforms
      - International distribution
   4. Specific content tags or warnings recommended
   5. Justification for your decision

   Format your response as a well-structured report with clear sections for each platform type.
"""
        explicitLanguageAnalysis = self.state.explicitLanguageAnalysis
        inappropriateThemeAnalysis = self.state.inappropriateThemeAnalysis
        misleadingContentAnalysis = self.state.misleadingContentAnalysis
        sentimentAnalysis = self.state.sentimentAnalysis

        mediaPlatformRecommendations_prompt = f"""## Explicit Language & Age-Appropriate Analysis Report:

{explicitLanguageAnalysis}

## Inappropriate Themes & Cultural Sensitivity Analysis Report:

{inappropriateThemeAnalysis}

## Misleading Content & Copyright Analysis Report:

{misleadingContentAnalysis}


## Sentiment & Brand Alignment Analysis Report:

{sentimentAnalysis}

"""
        messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": mediaPlatformRecommendations_prompt}
                ]
        llm = LLM(model="bedrock/us.amazon.nova-pro-v1:0")
        response = llm.call(messages=messages)
        return { "final_result" : response}

## Sample Script Content for Analysis

Define a sample TV/movie script that contains various elements requiring content moderation analysis. This crime drama script excerpt includes explicit language, violent themes, and mature content that will trigger responses from all four analysis agents, demonstrating the comprehensive moderation capabilities of our parallel workflow.


In [ ]:
script = """FADE IN:

   EXT. CITY STREET - NIGHT

   Rain pours down as DETECTIVE MORGAN (45, hardened) stands over a body covered by a sheet. Lightning flashes, illuminating the grim scene.

   DETECTIVE MORGAN
   (lighting a cigarette)
   Third one this month. Same damn signature.

   OFFICER JENKINS (30s) approaches, looking queasy.

   OFFICER JENKINS
   The mayor's going to have a fit. The press is already calling this guy "The Midnight Butcher."

   DETECTIVE MORGAN
   I don't give a damn what they call him. I'm going to catch this son of a bitch before he cuts up another girl.

   Morgan kneels down and pulls back the sheet, revealing a young woman's face, pale and lifeless.

   DETECTIVE MORGAN
   (whispering)
   I promise you justice, sweetheart. Whatever it takes.

   He stands, tosses his cigarette into a puddle.

   DETECTIVE MORGAN
   Jenkins, get me everything on the victim. And I mean everything. Where she worked, who she dated, what goddamn toothpaste she used.

   OFFICER JENKINS
   But sir, it's almost midnight and—

   DETECTIVE MORGAN
   (interrupting)
   Evil doesn't sleep, why should we?

   Morgan walks away as lightning flashes again, his silhouette momentarily frozen against the night sky.

   FADE OUT.
   """

## Execute Content Analysis Flow

Run the parallel content analysis workflow asynchronously on our sample script. This will trigger all four analysis agents to process the content simultaneously, each focusing on their specialized area of expertise, before aggregating the results into a comprehensive moderation report.


In [ ]:
response = await ContentAnalysisParallelFlow().kickoff_async(inputs = {
    "content" : script})

## Display Analysis Results

Print the comprehensive content moderation report generated by our parallel analysis workflow. The output includes the overall moderation decision, key concerns from each analysis agent, and platform-specific recommendations for different distribution channels.


In [ ]:
print(response)

## Summary
In this notebook, we successfully implemented a **Prompt Parallelization** workflow using CrewAI for content moderation in the media and entertainment industry. Here's what we built and demonstrated:

### **Specialized Analysis Agents**
1. **Explicit Language Analysis Agent** - Detected profanity, assessed age-appropriateness, and provided content ratings (G, PG, PG-13, R, etc.)
2. **Inappropriate Theme Analysis Agent** - Identified problematic themes, cultural sensitivity issues, and regional content concerns
3. **Misleading Content Analysis Agent** - Analyzed factual accuracy, copyright concerns, and legal compliance requirements
4. **Sentiment Analysis Agent** - Evaluated emotional tone, brand alignment, and potential audience impact

This implementation showcases how modern AI workflows can streamline content moderation processes while ensuring comprehensive analysis and compliance across multiple distribution platforms and global markets.
